In [2]:
import os
import pandas as pd


issues_dataset_path = "D:\Contributing\issue-analysis-eval-logs\data"
files = os.listdir(issues_dataset_path)
prefix = "so_"
files = [f for f in files if f.startswith(prefix)]

D:\Contributing\issue-analysis-eval-logs\data
